In [2]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from sklearn.linear_model import LinearRegression
import pickle
import unidecode
from bs4 import BeautifulSoup
import requests
import time
import matplotlib.pyplot as plt

In [34]:
# Need to find more accurate way to scrape active rosters

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.lineups.com/nba/depth-charts')
driver.refresh()
time.sleep(5)
html = driver.page_source
tables = pd.read_html(html)




====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST driver version for 97.0.4692
Driver [/Users/hsinger24/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


In [35]:
teams = ['Atlanta Hawks', 'Boston Celtics', 'Brooklyn Nets', 'Charlotte Hornets',
        'Chicago Bulls', 'Cleveland Cavaliers', 'Dallas Mavericks', 'Denver Nuggets',
        'Detroit Pistons', 'Golden State Warriors', 'Houston Rockets', 'Indiana Pacers',
        'Los Angeles Clippers', 'Los Angeles Lakers', 'Memphis Grizzlies', 'Miami Heat',
        'Milwaukee Bucks', 'Minnesota Timberwolves', 'New Orleans Pelicans',
        'New York Knicks', 'Oklahoma City Thunder', 'Orlando Magic', 'Philadelphia 76ers',
        'Phoenix Suns', 'Portland Trailblazers', 'Sacramento Kings', 'San Antonio Spurs',
        'Toronto Raptors', 'Utah Jazz', 'Washington Wizards']
def name_adjustment(x):
    try:
        names = x.split(' ')
        if len(names) == 4:
            name = names[0] + ' ' + names[1]
        if len(names) == 6:
            name = names[0] + ' ' + names[1] + ' ' + names[2]
        if len(names) == 5:
            name = names[0] + ' ' + names[1]
        if len(names) == 7:
            name = names[0] + ' ' + names[1] + ' ' + names[2]
    except:
        name = x
    return name

In [36]:
team_dict = {}
for team, table in zip(teams,tables):
    table.columns = table.columns.droplevel(0)
    for i in [1,2,3]:
        table[str(i)] = table[str(i)].apply(name_adjustment)
    table = table[['1', '2', '3']]
    players = list()
    for i in range(table.shape[0]): 
        for j in range(table.shape[1]):
            player = table.iloc[i, j]
            players = players + [player]
    team_dict[team] = players

['Trae', 'Young', '', 'T.', 'Young']
['Bogdan', 'Bogdanovic', '', 'B.', 'Bogdanovic']
['Kevin', 'Huerter', '', 'K.', 'Huerter']
['John', 'Collins', '', 'J.', 'Collins']
['Clint', 'Capela', '', 'C.', 'Capela']
['Delon', 'Wright', '', 'D.', 'Wright']
['Lou', 'Williams', '', 'L.', 'Williams']
["De'Andre", 'Hunter', '', 'D.', 'Hunter']
['Danilo', 'Gallinari', '', 'D.', 'Gallinari']
['Onyeka', 'Okongwu', '', 'O.', 'Okongwu']
['Sharife', 'Cooper', '', 'S.', 'Cooper']
['Timothe', 'Luwawu-Cabarrot', '', 'T.', 'Luwawu-Cabarrot']
['Kevin', 'Knox', 'II', '', 'K.', 'Knox', 'II']
['Jalen', 'Johnson', '', 'J.', 'Johnson']
['Gorgui', 'Dieng', '', 'G.', 'Dieng']
['Marcus', 'Smart', '', 'M.', 'Smart']
['Jaylen', 'Brown', '', 'J.', 'Brown']
['Jayson', 'Tatum', '', 'J.', 'Tatum']
['Robert', 'Williams', 'III', '', 'R.', 'Williams', 'III']
['Al', 'Horford', '', 'A.', 'Horford']
['Romeo', 'Langford', '', 'R.', 'Langford']
['Dennis', 'Schroder', '', 'D.', 'Schroder']
['Josh', 'Richardson', '', 'J.', 'Richard

['Théo', 'Maledon', '', 'T.', 'Maledon']
['Derrick', 'Favors', '', 'D.', 'Favors']
['Aleksej', 'Pokuševski', '', 'A.', 'Pokuševski']
['Cole', 'Anthony', '', 'C.', 'Anthony']
['Gary', 'Harris', '', 'G.', 'Harris']
['Franz', 'Wagner', '', 'F.', 'Wagner']
['Wendell', 'Carter', 'Jr.', '', 'W.', 'Carter', 'Jr.']
['Mo', 'Bamba', '', 'M.', 'Bamba']
['Jalen', 'Suggs', '', 'J.', 'Suggs']
['R.J.', 'Hampton', '', 'R.', 'Hampton']
['Terrence', 'Ross', '', 'T.', 'Ross']
['Chuma', 'Okeke', '', 'C.', 'Okeke']
['Robin', 'Lopez', '', 'R.', 'Lopez']
['Ignas', 'Brazdeikis', '', 'I.', 'Brazdeikis']
['Admiral', 'Schofield', '', 'A.', 'Schofield']
['Moritz', 'Wagner', '', 'M.', 'Wagner']
['Tyrese', 'Maxey', '', 'T.', 'Maxey']
['Furkan', 'Korkmaz', '', 'F.', 'Korkmaz']
['Danny', 'Green', '', 'D.', 'Green']
['Tobias', 'Harris', '', 'T.', 'Harris']
['Joel', 'Embiid', '', 'J.', 'Embiid']
['Matisse', 'Thybulle', '', 'M.', 'Thybulle']
['Seth', 'Curry', '', 'S.', 'Curry']
['Shake', 'Milton', '', 'S.', 'Milton']
['